In [1]:
!pip install PyGreSQL
# !pip install nbimporter

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import csv
import pgdb
import pickle
import zipfile
# import nbimporter
import numpy as np
import pandas as pd
import scipy
# import ProviderPredictionScript

from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.multioutput import MultiOutputClassifier

In [3]:
### How many providers do we want to synthesize?

## User defined for each specialty
num_providers = 1000

In [4]:
### Probability of number of claims per provider

### We'll eventually bring inference (of number of claims for each provider) from all customers for each specialty.

### Based on MonteCarlo simulation from the empirical data
num_claims_df = pd.read_csv('num_claims_df.csv')


num_iter = 100000
num_claims_avg_lst = []
for ii in range(0, num_iter):
    num_claims_avg_lst.append(round(num_claims_df.sample(round(0.1*len(num_claims_df))).num_claims.mean()))
dist_mean = np.round(np.mean(num_claims_avg_lst))
dist_std = np.round(np.std(num_claims_avg_lst))
group = [int(round(x)) for x in np.random.normal(loc=dist_mean, scale=3*dist_std,size=num_providers)]
scipy.stats.describe(group)

DescribeResult(nobs=1000, minmax=(47, 222), mean=134.641, variance=875.1933123123124, skewness=0.0801505428566701, kurtosis=-0.18459367975360053)

In [5]:
### Determine probability of different combinations 
### We'll eventually bring inference (of number of claims for each combination) from all customers for each specialty.
all_combinations = pd.read_csv('all_combinations_all.csv')
sel_combinations = all_combinations[all_combinations.prob_claims>=0.01]

In [6]:
sel_combinations

,combos,place_of_service,pat_gender,diag_values,num_claims,prob_claims
2355,90785;WD:90837;WD,30,None,F411,918,0.011822
2414,90785;WD:90837;WD,30,None,F4323,1391,0.017913
5548,90791;GT@WD,22,F,F411,1036,0.013341
5705,90791;GT@WD,22,F,F4323,832,0.010714
9143,90791;WD,30,F,F411,1852,0.023849
...,...,...,...,...,...,...
71225,90847;WD,30,None,F4320,918,0.011822
71248,90847;WD,30,None,F4322,970,0.012491
71261,90847;WD,30,None,F4323,2221,0.028601
72500,90853;WC,30,None,F1120,1489,0.019175


In [7]:
## Unique combinations and corresponding weights
comb_list = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).values
weights = sel_combinations['prob_claims'].tolist()

In [8]:
import random  
final_df_list = []
random_outliers = random.sample(range(0, num_providers), int(round(0.01*num_providers)))
for ii in range(0, num_providers):
    if ii in random_outliers:
        tmp = pd.DataFrame(random.choices(comb_list, random.shuffle(weights), k=group[ii]), 
                       columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
    else:
        tmp = pd.DataFrame(random.choices(comb_list, weights, k=group[ii]), 
                       columns = sel_combinations.drop(['num_claims', 'prob_claims'], axis = 1).columns)
    tmp['ren_id'] = 'prov_' + str(ii)
    tmp['prov_claim_id'] = ['claim_' + str(x) for x in range(0, group[ii])]
    final_df_list.append(tmp)
final_df = pd.concat(final_df_list)

In [9]:
final_df[final_df['ren_id'] == 'prov_0']

,combos,place_of_service,pat_gender,diag_values,ren_id,prov_claim_id
0,90837;WD,30,None,F331,prov_0,claim_0
1,90837;WD,30,M,F411,prov_0,claim_1
2,90837;WD,30,F,F4320,prov_0,claim_2
3,90837;WD,30,F,F411,prov_0,claim_3
4,90837;GT@WD,22,F,F411,prov_0,claim_4
...,...,...,...,...,...,...
160,90837;WD,22,F,F411,prov_0,claim_160
161,90837;WD,30,None,F321,prov_0,claim_161
162,90837;95@WD,22,F,F331,prov_0,claim_162
163,90837;WD,30,None,F4323,prov_0,claim_163


In [11]:
a = []

for index, row in final_df.iterrows():  
    b = (row['combos'].split(":"))
    a.append(b)

final_df["combos"] = a
final_df

,combos,place_of_service,pat_gender,diag_values,ren_id,prov_claim_id
0,[90837;WD],30,None,F331,prov_0,claim_0
1,[90837;WD],30,M,F411,prov_0,claim_1
2,[90837;WD],30,F,F4320,prov_0,claim_2
3,[90837;WD],30,F,F411,prov_0,claim_3
4,[90837;GT@WD],22,F,F411,prov_0,claim_4
...,...,...,...,...,...,...
124,[90837;WD],30,F,F320,prov_999,claim_124
125,[90834;WD],30,F,F331,prov_999,claim_125
126,[90834;WD],30,None,F330,prov_999,claim_126
127,[90834;95@WD],22,M,F4323,prov_999,claim_127


In [12]:
final_df = final_df.explode('combos')
a = []

for index, row in final_df.iterrows():  
    b = (row['combos'].split(";"))
    a.append(b)
final_df["combos"] = a
final_df

,combos,place_of_service,pat_gender,diag_values,ren_id,prov_claim_id
0,"[90837, WD]",30,None,F331,prov_0,claim_0
1,"[90837, WD]",30,M,F411,prov_0,claim_1
2,"[90837, WD]",30,F,F4320,prov_0,claim_2
3,"[90837, WD]",30,F,F411,prov_0,claim_3
4,"[90837, GT@WD]",22,F,F411,prov_0,claim_4
...,...,...,...,...,...,...
124,"[90837, WD]",30,F,F320,prov_999,claim_124
125,"[90834, WD]",30,F,F331,prov_999,claim_125
126,"[90834, WD]",30,None,F330,prov_999,claim_126
127,"[90834, 95@WD]",22,M,F4323,prov_999,claim_127


In [13]:
final_df[['hcpcs_code','mod_values']] = pd.DataFrame(final_df.pop('combos').values.tolist(), index=final_df.index)
final_df

,place_of_service,pat_gender,diag_values,ren_id,prov_claim_id,hcpcs_code,mod_values
0,30,None,F331,prov_0,claim_0,90837,WD
1,30,M,F411,prov_0,claim_1,90837,WD
2,30,F,F4320,prov_0,claim_2,90837,WD
3,30,F,F411,prov_0,claim_3,90837,WD
4,22,F,F411,prov_0,claim_4,90837,GT@WD
...,...,...,...,...,...,...,...
124,30,F,F320,prov_999,claim_124,90837,WD
125,30,F,F331,prov_999,claim_125,90834,WD
126,30,None,F330,prov_999,claim_126,90834,WD
127,22,M,F4323,prov_999,claim_127,90834,95@WD


In [14]:
final_df["combo"] = final_df["hcpcs_code"] +"@"+final_df["mod_values"]
final_df

,place_of_service,pat_gender,diag_values,ren_id,prov_claim_id,hcpcs_code,mod_values,combo
0,30,None,F331,prov_0,claim_0,90837,WD,90837@WD
1,30,M,F411,prov_0,claim_1,90837,WD,90837@WD
2,30,F,F4320,prov_0,claim_2,90837,WD,90837@WD
3,30,F,F411,prov_0,claim_3,90837,WD,90837@WD
4,22,F,F411,prov_0,claim_4,90837,GT@WD,90837@GT@WD
...,...,...,...,...,...,...,...,...
124,30,F,F320,prov_999,claim_124,90837,WD,90837@WD
125,30,F,F331,prov_999,claim_125,90834,WD,90834@WD
126,30,None,F330,prov_999,claim_126,90834,WD,90834@WD
127,22,M,F4323,prov_999,claim_127,90834,95@WD,90834@95@WD


providers area is alphereta with further away having fewer paitients<br>
<b>work on cost </b><br>
send me a sample members file ecuding phi using age to randomize the date

In [15]:
cost = pd.read_csv("line_amt_df.csv", dtype={'place_of_service': str}, encoding= 'unicode_escape')
cost['hcpcs_code'] = cost['hcpcs_code'].astype(str)
cost['mod_values'] = cost['mod_values'].astype(str)
cost

,hcpcs_code,proc_values,diag_values,mod_values,place_of_service,pat_gender,amount_charged,amount_paid
0,90837,90837,F900,95@WD,22,M,120.0,89.70
1,90837,90837,F331@F4312,95@WD,22,M,192.0,89.70
2,90837,90837,F411,WD,30,F,150.0,89.70
3,90837,90837@90837,F411@Z630,95@WD,22,F,150.0,100.63
4,90837,90837@90837,F411@Z630,95@WD,22,F,150.0,100.63
...,...,...,...,...,...,...,...,...
1671884,90837,90837@90837@90837@90837@90837@90837@90837@90837,F411,WD,30,F,65.0,65.00
1671885,90837,90837@90837@90837@90837@90837,F411,WD,30,M,65.0,0.00
1671886,90834,90834@90834@90834@90834@90834@90834@90834@9083...,F411,WD,30,F,125.0,0.00
1671887,90837,90837@90837,F331,WD,30,F,150.0,0.00


In [16]:
a = []
for index, row in cost.iterrows():  
    b = (row['hcpcs_code'].split("@")+ row["mod_values"].split("@"))
    b.sort()
    b = [str(x) for x in b]
    b = "@".join(b)
    a.append(b)

cost["combo"] = a
cost = cost.drop(columns=['diag_values', 'place_of_service',"pat_gender", "mod_values", "proc_values"])
cost.head()

,hcpcs_code,amount_charged,amount_paid,combo
0,90837,120.0,89.70,90837@95@WD
1,90837,192.0,89.70,90837@95@WD
2,90837,150.0,89.70,90837@WD
3,90837,150.0,100.63,90837@95@WD
4,90837,150.0,100.63,90837@95@WD


In [17]:
cost[(cost['combo'] == '90837@WD') & (cost['amount_paid'] > 0)]

,hcpcs_code,amount_charged,amount_paid,combo
2,90837,150.0,89.70,90837@WD
5,90837,225.0,163.00,90837@WD
13,90837,110.0,89.70,90837@WD
15,90837,110.0,98.19,90837@WD
16,90837,110.0,98.19,90837@WD
...,...,...,...,...
1671865,90837,125.0,96.02,90837@WD
1671868,90837,100.0,89.70,90837@WD
1671872,90837,150.0,89.70,90837@WD
1671875,90837,115.0,89.70,90837@WD


In [18]:
c = cost.groupby(['combo'])['amount_charged'].sum()
c = c.reset_index()
e = cost.groupby(['combo'],as_index = False).size()
c["count"] = e["size"]
c["avg_charge"] = c["amount_charged"]/ c["count"]


ds = cost[cost.amount_paid !=0]
d = ds.groupby(['combo'])['amount_paid'].sum()
d = d.reset_index()
e = ds.groupby(['combo'], as_index= False).size()
d["count"] = e["size"]
d = d.merge(c[["combo", "avg_charge"]])
d["avg_paid_pct"] = (d["amount_paid"]/ d["count"])/d["avg_charge"]


f = cost.groupby(['combo'])['amount_paid'].sum()
f = f.reset_index()
e = cost.groupby(['combo'], as_index=False).size()
# e = e.reset_index()
f["count"] = e["size"]
f = f.merge(c[["combo", "avg_charge"]])
f["avg_paid_pct"] = (f["amount_paid"]/ f["count"])/f["avg_charge"]
f = f[f.avg_paid_pct == 0.000000]

d = d.append(f)

c

,combo,amount_charged,count,avg_charge
0,00145@QS@QX@WJ,1100.00,1,1100.000000
1,05@90837@WD,185.00,1,185.000000
2,1000F@None,0.01,1,0.010000
3,1036F@None,0.00,29,0.000000
4,1101F@None,0.50,3,0.166667
...,...,...,...,...
1419,T1013@UA@XC,120.00,2,60.000000
1420,T1013@XC,120.00,2,60.000000
1421,T1016@UB@W3,627.30,4,156.825000
1422,T1019@U5,312.08,6,52.013333


In [19]:
scb = 0#50      #may want to change scale so that it is more in line with reality
sca = .05

cost_dict = c.set_index('combo').to_dict()
final_df['cost'] = final_df['combo'].apply(lambda x: cost_dict["avg_charge"][x])
final_df['real charge'] = np.random.normal(final_df['cost'], scale = scb).astype(int)
final_df['real charge'] = final_df['real charge'].abs()
final_df = final_df.drop(columns=['cost'])


paid_dict = d.set_index('combo').to_dict()
final_df['paid'] = final_df['combo'].apply(lambda x: paid_dict["avg_paid_pct"][x])
final_df['real paid'] = np.random.normal(final_df['paid'], scale = sca)*final_df["real charge"].astype(int)
final_df['real paid'] = final_df['real paid'].abs()

final_df = final_df.drop(columns=['paid'])
final_df["real paid"] = final_df["real paid"].astype(int)

final_df.to_csv("final.csv")
#final_df = final_df.drop(columns=['combo'])

final_df

,place_of_service,pat_gender,diag_values,ren_id,prov_claim_id,hcpcs_code,mod_values,combo,real charge,real paid
0,30,None,F331,prov_0,claim_0,90837,WD,90837@WD,146,105
1,30,M,F411,prov_0,claim_1,90837,WD,90837@WD,146,107
2,30,F,F4320,prov_0,claim_2,90837,WD,90837@WD,146,107
3,30,F,F411,prov_0,claim_3,90837,WD,90837@WD,146,117
4,22,F,F411,prov_0,claim_4,90837,GT@WD,90837@GT@WD,149,99
...,...,...,...,...,...,...,...,...,...,...
124,30,F,F320,prov_999,claim_124,90837,WD,90837@WD,146,104
125,30,F,F331,prov_999,claim_125,90834,WD,90834@WD,127,62
126,30,None,F330,prov_999,claim_126,90834,WD,90834@WD,127,80
127,22,M,F4323,prov_999,claim_127,90834,95@WD,90834@95@WD,129,70


In [20]:
final_df["claim"] = final_df["prov_claim_id"]+"@"+final_df["ren_id"]
final_df_1 = final_df
final_df_1.groupby(['claim']).agg({'real charge': sum, 'real paid': sum}).reset_index()
final_df_1


,place_of_service,pat_gender,diag_values,ren_id,prov_claim_id,hcpcs_code,mod_values,combo,real charge,real paid,claim
0,30,None,F331,prov_0,claim_0,90837,WD,90837@WD,146,105,claim_0@prov_0
1,30,M,F411,prov_0,claim_1,90837,WD,90837@WD,146,107,claim_1@prov_0
2,30,F,F4320,prov_0,claim_2,90837,WD,90837@WD,146,107,claim_2@prov_0
3,30,F,F411,prov_0,claim_3,90837,WD,90837@WD,146,117,claim_3@prov_0
4,22,F,F411,prov_0,claim_4,90837,GT@WD,90837@GT@WD,149,99,claim_4@prov_0
...,...,...,...,...,...,...,...,...,...,...,...
124,30,F,F320,prov_999,claim_124,90837,WD,90837@WD,146,104,claim_124@prov_999
125,30,F,F331,prov_999,claim_125,90834,WD,90834@WD,127,62,claim_125@prov_999
126,30,None,F330,prov_999,claim_126,90834,WD,90834@WD,127,80,claim_126@prov_999
127,22,M,F4323,prov_999,claim_127,90834,95@WD,90834@95@WD,129,70,claim_127@prov_999


In [21]:
final_df["total charged"] = final_df_1["real charge"]
final_df["total paid"] = final_df_1["real paid"]
final_df = final_df.drop(columns=['claim'])
final_df.to_csv("final_df.csv")
final_df

,place_of_service,pat_gender,diag_values,ren_id,prov_claim_id,hcpcs_code,mod_values,combo,real charge,real paid,total charged,total paid
0,30,None,F331,prov_0,claim_0,90837,WD,90837@WD,146,105,146,105
1,30,M,F411,prov_0,claim_1,90837,WD,90837@WD,146,107,146,107
2,30,F,F4320,prov_0,claim_2,90837,WD,90837@WD,146,107,146,107
3,30,F,F411,prov_0,claim_3,90837,WD,90837@WD,146,117,146,117
4,22,F,F411,prov_0,claim_4,90837,GT@WD,90837@GT@WD,149,99,149,99
...,...,...,...,...,...,...,...,...,...,...,...,...
124,30,F,F320,prov_999,claim_124,90837,WD,90837@WD,146,104,146,104
125,30,F,F331,prov_999,claim_125,90834,WD,90834@WD,127,62,127,62
126,30,None,F330,prov_999,claim_126,90834,WD,90834@WD,127,80,127,80
127,22,M,F4323,prov_999,claim_127,90834,95@WD,90834@95@WD,129,70,129,70
